<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/dataset/txt_to_json_to_corpus_iirc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

# txt 파일 경로와 생성할 json 파일 경로 설정
input_txt_file = '/content/유형4_v2.txt'  # 질문들이 저장된 텍스트 파일 경로
output_json_file = '/content/questions4_v2.json'  # 결과 JSON 파일 경로

# 텍스트 파일에서 질문들을 읽어오기
with open(input_txt_file, 'r', encoding='utf-8') as f:
    questions = f.read().splitlines()

# 각 질문을 JSON 형식으로 변환
json_data = []
for question in questions:
    if question.strip():  # 빈 줄 무시
        json_data.append({
            "type": "4",
            "dataset": "IIRC",
            "question": question.strip()
        })

# JSON 파일로 저장
with open(output_json_file, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print("Questions have been saved to", output_json_file)


Questions have been saved to /content/questions4_v2.json


## Answer list matching

In [ ]:
import json

# Load questions1.json and iirc.json data
with open('/content/questions1_v2.json', 'r') as file:
    questions1_data = json.load(file)

with open('/content/IIRC_dataset (1).json', 'r') as file:
    iirc_data = json.load(file)

# Create a dictionary to easily access iirc questions based on the question text
iirc_dict = {}
for entry in iirc_data:
    for question_entry in entry.get("questions", []):
        question_text = question_entry.get("question")
        if question_text:
            iirc_dict[question_text] = question_entry

# Process and combine data
combined_data = []
for question_entry in questions1_data:
    question_text = question_entry.get("question")
    iirc_match = iirc_dict.get(question_text)

    if iirc_match:
        # Extract answer safely, checking for 'answer_spans' presence and data
        answer_spans = iirc_match.get("answer", {}).get("answer_spans", [])
        answer = answer_spans[0].get("text", "No answer found") if answer_spans else "No answer found"

        # Extract context and question_links, handling missing data
        context = iirc_match.get("context", [])
        question_links = iirc_match.get("question_links", [])

        # Structure evidence_list with context and question_links in separate sections
        evidence_list = {
            "context": context,
            "question_links": question_links
        }
    else:
        # If no match, set answer to "helpme" and provide an empty evidence_list structure
        answer = "helpme"
        evidence_list = {
            "context": [],
            "question_links": []
        }

    # Construct the final JSON structure
    combined_entry = {
        "type": question_entry.get("type"),
        "dataset": question_entry.get("dataset"),
        "question": question_text,
        "answer": answer,
        "evidence_list": evidence_list
    }
    combined_data.append(combined_entry)

# Save the output JSON
with open('/content/combined_output.json', 'w') as file:
    json.dump(combined_data, file, indent=4)

print("Combined JSON has been saved to /content/combined_output_v2.json")


Combined JSON has been saved to /content/combined_output_v2.json


In [ ]:
######################
#answer 찾는법 수정
import json

# 파일 경로 설정
combined_file = '/content/combined_output.json'  # combined_output.json 파일 경로
iirc_file = '/content/IIRC_dataset (1).json'  # iirc.json 파일 경로
output_file = '/content/updated_combined_output_v2.json'  # 수정된 데이터를 저장할 파일 경로

# 파일 로드
with open(combined_file, 'r', encoding='utf-8') as f:
    combined_data = json.load(f)

with open(iirc_file, 'r', encoding='utf-8') as f:
    iirc_data = json.load(f)

# IIRC 데이터에서 질문에 대한 답을 찾는 함수
def find_answer_in_iirc(question, iirc_data):
    for entry in iirc_data:
        for q in entry.get('questions', []):
            if q.get('question') == question:
                answer_info = q.get('answer')
                answer_type = answer_info.get('type')

                # type에 따른 답을 추출
                if answer_type == "span":
                    answer_text = answer_info.get('answer_spans')[0].get('text') if answer_info.get('answer_spans') else "No answer found"
                elif answer_type == "value" or answer_type == "binary":
                    answer_text = answer_info.get('answer_value', "No answer found")
                else:
                    answer_text = "No answer found"

                return answer_text
    return "No answer found"

# combined_output 데이터 업데이트
for entry in combined_data:
    if entry.get('answer') == "No answer found":
        question = entry.get('question')
        updated_answer = find_answer_in_iirc(question, iirc_data)
        entry['answer'] = updated_answer

# 수정된 데이터를 새로운 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print("Updated combined_output has been saved to", output_file)

Updated combined_output has been saved to /content/updated_combined_output_v2.json


## Create urls

In [ ]:
!pip install openai
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl
import openai
import time
from tqdm import tqdm
!pip install openai==0.28
import json

In [ ]:
from google.colab import userdata, drive
# For Hugging Face Hub setting
from huggingface_hub import login

# Upload the HuggingFace token (should have WRITE access) from Colab secrets
HF = userdata.get('HF')

# This is needed to upload the model to HuggingFace
login(token=HF,add_to_git_credential=True)
import openai
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# 질문에서 엔터티를 추출하는 함수
def get_entities(query):
    messages = [
        {
            "role": "system",
            "content": """
            Extract all the named entities from the following question. Don't print anything other than the entities.
            ex)
            question: What was the father of Kasper Schmeichel voted to be by the IFFHS in 1992?
            ->
            Kasper Schmeichel,IFFHS,1992
            """
        },
        {
            "role": "user",
            "content": query
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message['content']

# JSON 파일 경로 설정
input_file = '/content/updated_combined_output.json'  # 기존 JSON 파일 경로
output_file = '/content/iirc_class_with_context_url.json'  # 결과 JSON 파일 경로

# 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터 변환 및 context_url 추가
new_data = []
for entry in data:
    question = entry['question']
    entities = get_entities(question)  # LLM으로부터 엔터티 추출
    entity_list = [entity.strip() for entity in entities.split(',')]  # 엔터티를 리스트로 변환

    # 엔터티를 기반으로 context_url 생성
    context_url = [f"https://en.wikipedia.org/wiki/{entity.replace(' ', '_')}" for entity in entity_list]

    # 새로운 엔트리 구성
    new_entry = {
        "type": entry['type'],
        "dataset": entry['dataset'],
        "question": question,
        "context_url": context_url,
        "answer": entry.get('answer', "helpme"),  # 기존 answer를 가져오거나 없으면 'helpme'
        "evidence_list": entry.get('evidence_list', [])  # 기존 evidence_list를 가져오거나 빈 리스트
    }
    new_data.append(new_entry)

# 결과 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

print("The data with context_url has been saved to", output_file)

The data with context_url has been saved to /content/iirc_class_with_context_url.json


In [ ]:
###############
#상위 title url 추가
import json

# Load questions1.json and iirc.json data
with open('/content/iirc_class_with_context_url.json', 'r') as file:
    questions1_data = json.load(file)

with open('/content/IIRC_dataset (1).json', 'r') as file:
    iirc_data = json.load(file)

# Create a dictionary to easily access iirc questions and their title based on the question text
iirc_dict = {}
for entry in iirc_data:
    title = entry.get("title", "")
    for question_entry in entry.get("questions", []):
        question_text = question_entry.get("question")
        if question_text:
            iirc_dict[question_text] = {"question_entry": question_entry, "title": title}

# Process and combine data
combined_data = []
for question_entry in questions1_data:
    question_text = question_entry.get("question")
    iirc_match = iirc_dict.get(question_text)

    if iirc_match:
        # Extract answer safely, checking for 'answer_spans' presence and data
        answer_spans = iirc_match["question_entry"].get("answer", {}).get("answer_spans", [])
        answer = answer_spans[0].get("text", "No answer found") if answer_spans else "No answer found"

        # Extract context and question_links, handling missing data
        context = iirc_match["question_entry"].get("context", [])
        question_links = iirc_match["question_entry"].get("question_links", [])

        # Add the title to question_links if it's not already present
        if iirc_match["title"] and iirc_match["title"] not in question_links:
            question_links.append(iirc_match["title"])

        # Structure evidence_list with context and question_links in separate sections
        evidence_list = {
            "context": context,
            "question_links": question_links
        }
    else:
        # If no match, set answer to "helpme" and provide an empty evidence_list structure
        answer = "helpme"
        evidence_list = {
            "context": [],
            "question_links": []
        }

    # Construct the final JSON structure
    combined_entry = {
        "type": question_entry.get("type"),
        "dataset": question_entry.get("dataset"),
        "question": question_text,
        "context_url": question_entry.get("context_url", []),
        "answer": answer,
        "evidence_list": evidence_list
    }
    combined_data.append(combined_entry)

# Save the output JSON
with open('/content/combined_output.json', 'w') as file:
    json.dump(combined_data, file, indent=4)

print("Combined JSON has been saved to /content/combined_output.json")


Combined JSON has been saved to /content/combined_output.json


In [ ]:
###############
#정답 url 필드 추가
import json

# Load the IIRC_4type_classification_title.json data
with open('/content/IIRC_4type_classification_title.json', 'r') as file:
    data = json.load(file)

# Base URL for constructing answer URLs
base_url = "https://en.wikipedia.org/wiki/"

# Process each entry to add answer_url field
for entry in data:
    question_links = entry.get("evidence_list", {}).get("question_links", [])

    # Construct answer_url list by concatenating base URL with each link
    answer_url = [base_url + link.replace(" ", "_") for link in question_links]

    # Add the answer_url field to the entry
    entry["answer_url"] = answer_url

# Save the modified data to a new JSON file
with open('/content/IIRC_4type_classification_title_with_urls.json', 'w') as file:
    json.dump(data, file, indent=4)

print("Updated JSON has been saved to /content/IIRC_4type_classification_title_with_urls.json")


Updated JSON has been saved to /content/IIRC_4type_classification_title_with_urls.json


## Compose corpus

In [ ]:
#######################
## 코퍼스 구축
import json
import requests
from bs4 import BeautifulSoup

def clean_wikipedia_content(url):
    # Send a GET request to fetch the content of the page
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Remove references, scripts, styles, and other irrelevant elements
    for element in soup(['sup', 'img', 'style', 'script']):
        element.decompose()

    # Extract the main content from bodyContent div
    content_div = soup.find('div', {'id': 'bodyContent'})
    paragraphs = content_div.find_all('p')

    # Extract the text from paragraphs and join them with spaces
    text_content = ' '.join([para.get_text(strip=True) for para in paragraphs])

    # Clean the text (remove references like [1], [2], extra whitespaces, etc.)
    clean_text = text_content.replace("\n", " ").strip()

    return clean_text

# Load the JSON file and extract text content from context and answer URLs
def extract_context_texts(json_file, output_file):
    with open(json_file, 'r') as infile:
        data = json.load(infile)

    # Set to keep track of already visited URLs to avoid duplication
    visited_urls = set()

    with open(output_file, 'w') as outfile:
        for entry in data:
            # Combine context_url and answer_url
            urls = entry.get('context_url', []) + entry.get('answer_url', [])

            for url in urls:
                if url not in visited_urls:
                    visited_urls.add(url)
                    try:
                        # Log the progress
                        print(f"Fetching URL: {url}")

                        # Clean and fetch content
                        text = clean_wikipedia_content(url)
                        outfile.write(f"{text}\n")  # Save each page's text on a single line

                    except Exception as e:
                        print(f"Error fetching {url}: {e}")

# Call the function with the specified JSON file and output .txt file
extract_context_texts('/content/IIRC_4type_classification_500_v2.json', '/content/extracted_texts_iirc.txt')


Fetching URL: https://en.wikipedia.org/wiki/Bain
Fetching URL: https://en.wikipedia.org/wiki/Wlad_Godzich
Fetching URL: https://en.wikipedia.org/wiki/University_of_Geneva
Fetching URL: https://en.wikipedia.org/wiki/Thomas_Bain_(Orange)
Fetching URL: https://en.wikipedia.org/wiki/McCarrick
Fetching URL: https://en.wikipedia.org/wiki/1993
Fetching URL: https://en.wikipedia.org/wiki/Dublin
Fetching URL: https://en.wikipedia.org/wiki/Annie_McCarrick
Fetching URL: https://en.wikipedia.org/wiki/Blackjack
Fetching URL: https://en.wikipedia.org/wiki/Blackjack_(American_band)
Fetching URL: https://en.wikipedia.org/wiki/Peter_Frampton
Fetching URL: https://en.wikipedia.org/wiki/Sandy_Gennaro
Fetching URL: https://en.wikipedia.org/wiki/Shires
Fetching URL: https://en.wikipedia.org/wiki/1941
Fetching URL: https://en.wikipedia.org/wiki/History_of_the_Cleveland_Rams
Fetching URL: https://en.wikipedia.org/wiki/Abe_Shires
Fetching URL: https://en.wikipedia.org/wiki/NFL
Fetching URL: https://en.wikiped